# Vehicle Detection #

** Main video pipeline for the Vehicle Detection project **

#### Imports ####

In [1]:
import numpy as np
import pickle
import cv2
import matplotlib.pyplot as plt
from skimage.feature import hog
from scipy.ndimage.measurements import label
import time
from moviepy.editor import VideoFileClip

%matplotlib inline

#### Utility Funtions ####

In [8]:
### Code taken from Udacity's lessons on SDC nanodegree

def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2HLS':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    if conv == 'BGR2LUV':
        return cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
    if conv == 'BGR2HLS':
        return cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    if conv == 'BGR2HSV':
        return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec, block_norm='L2')
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec, block_norm='L2')
        return features

### Get resized and then flattened, concatenated version of the original image
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
### Get Histogram of the image
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

### For a given scale and search section, return list of boxes where car is detected
def find_cars(img, ystart, ystop, scale, svc, X_scaler, pca, orient, pix_per_cell, 
              cell_per_block, spatial_size, hist_bins, threshold=2, color_space='BGR2HLS'):
    
    draw_img = np.copy(img)
  #  img = img.astype(np.float32)/255

    bbox_list = []
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = convert_color(img_tosearch, conv=color_space)
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features 
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
            #Apply PCA for feature set reduction    
            test_features = pca.transform(test_features)
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                bbox_list.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)))

    return bbox_list

### Merge boxes: Create heatmap and Apply threshold; Return label boxes 
def combine_bbox(img, bbox_list, threshold, grow_boxes=False, grow_by=10):
    if grow_boxes:
        bbox_list_exp = []
        for box in bbox_list:
            bbox_list_exp.append(( (box[0][0]-grow_by, box[0][1]), (box[1][0]+grow_by, box[1][1]) ))
        bbox_list = bbox_list_exp
    
    heatmap = add_heat(np.zeros_like(img), bbox_list)
    heatmap = apply_threshold(heatmap, threshold)
    labels = label(heatmap)
    bboxes = get_labeled_bboxes(img, labels)
        
    return bboxes
    
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

def apply_threshold(heatmap, threshold=2):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

### Seperate out each detection into its own set of boxes
def get_labeled_bboxes(img, labels):
    bbox = []
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox.append(((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy))))
    # Return the image
    return bbox

### Draw given rectangles on given image
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    draw_img = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(draw_img, tuple(bbox[0]), tuple(bbox[1]), color, thick)
    # Return the image copy with boxes drawn
    return draw_img

### Calculate average box coordinates for given boxes
def get_average(img, current_bbox, previous_bbox):
    if previous_bbox == None or current_bbox == None or len(current_bbox) != len(previous_bbox):
        return current_bbox
    
    # group boxes in similar locations together using overlap counter
    overlap_counter = []
    avg_bbox =[]
    heatmap = np.zeros_like(img)
    for cbbox in current_bbox:
        overlap_counter = []
        for pbbox in previous_bbox:
            heatmap[:] = 0
            heatmap[cbbox[0][1]:cbbox[1][1], cbbox[0][0]:cbbox[1][0]] += 1
            heatmap[pbbox[0][1]:pbbox[1][1], pbbox[0][0]:pbbox[1][0]] += 1
            heatmap[heatmap < 2] = 0
            overlap_counter.append(np.sum(heatmap))
        max_ovl = np.argmax(np.array(overlap_counter))
        avg = (np.array(cbbox) + np.array(previous_bbox[max_ovl])) // 2
        avg_bbox.append(avg.tolist())
        
    return avg_bbox        


#### Parameters for the pipeline: ####

In [9]:
### Initialize feature extraction parameters and import classifer & scaler
color_space='RGB2HLS'
spatial_size=(32, 32)
hist_bins=16
orient=9
pix_per_cell=6
cell_per_block=2
hog_channel='ALL'
spatial_feat=True
hist_feat=True
hog_feat=True

#### Import the Classifier, Scaler and PCA object from pickle file: ####

In [10]:
with open('svc_pca_scaler.p', 'rb') as f:
    svc_dict = pickle.load(f)
clf_svc_pca = svc_dict['clf_svc_pca']
X_scaler = svc_dict['scaler']
pca = svc_dict['pca']

#### Image Processing Pipeline: ####

In [17]:
frame_box_list = []
n_frames = 20
last_bbox = None
prev_frame_num1 = 20
prev_frame_num2 = 10

def process_img(img):
    bbox_current = find_cars(img, ystart=400, ystop=550, scale=1.5, svc=clf_svc_pca, X_scaler=X_scaler, pca=pca, orient=orient, 
                      pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, spatial_size=spatial_size, 
                      hist_bins=hist_bins, threshold=2, color_space=color_space)
    
    bbox2 = find_cars(img, ystart=415, ystop=500, scale=0.8, svc=clf_svc_pca, X_scaler=X_scaler, pca=pca, orient=orient, 
                      pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, spatial_size=spatial_size, 
                      hist_bins=hist_bins, threshold=2, color_space=color_space)
    bbox_current.extend(bbox2)
    labelled_box = combine_bbox(img, bbox_current, threshold=2)
          
    global frame_box_list
    global last_bbox
    
    if len(labelled_box) > 0:
        if len(frame_box_list) > n_frames:
            frame_box_list[:-1] = frame_box_list[1:]
            frame_box_list[-1] = labelled_box
        else:
            frame_box_list.append(labelled_box)
    
    if len(frame_box_list) >= 4:
        timelapsed_box_list = []
        for boxes in frame_box_list:
            timelapsed_box_list.extend(boxes)
        combined_label_box = combine_bbox(img, timelapsed_box_list, threshold=len(frame_box_list)-1, grow_boxes=True)
    else:
        combined_label_box = combine_bbox(img, labelled_box, threshold=2)
        
    draw_img = draw_boxes(img, combined_label_box, color=(0, 255, 100))
    
        ### PIP for troubleshooting
    #mini_img = draw_boxes(img, bbox_current, color=(255,0,0))
    #mini_img = cv2.resize(mini_img, (mini_img.shape[1]//3, mini_img.shape[0]//3 ))
    #draw_img[:mini_img.shape[0], :mini_img.shape[1]] = mini_img
    
    #mini_img2 = draw_boxes(img, bbox_prev, color=(255,0,200))
    #mini_img2 = cv2.resize(mini_img2, (mini_img2.shape[1]//3, mini_img2.shape[0]//3 ))
    #offset = mini_img.shape[1] + 10
    #draw_img[:mini_img2.shape[0], offset:offset+mini_img2.shape[1]] = mini_img2

    return draw_img

#### Apply to video: ####

In [18]:
video1 = 'project_video.mp4'
video1_output = 'project_video_output_6.mp4'
clip1 = VideoFileClip(video1)#.subclip(20,24)
imgs = clip1.fl_image(process_img)
%time imgs.write_videofile(video1_output, audio=False)

[MoviePy] >>>> Building video project_video_output_6.mp4
[MoviePy] Writing video project_video_output_6.mp4


100%|███████████████████████████████████████████████████████████████████████████▉| 1260/1261 [2:48:32<00:08,  8.51s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output_6.mp4 

Wall time: 2h 48min 33s
